In [ ]:
require 'nn'    --import nn packages

### *up-projection*
<p> ![](https://raw.githubusercontent.com/tonioecto/psc-profondeur/model_version_1/images/up_projection.png)
<p> $owidth  = (width  - 1) * dW - 2*padW + kW + adjW$
<p> $oheight = (height - 1) * dH - 2*padH + kH + adjH$
<p> module = nn.SpatialReflectionPadding(padLeft, padRight, padTop, padBottom)
<p> module = nn.SpatialFullConvolution(nInputPlane, nOutputPlane, kW, kH, [dW], [dH], [padW], [padH], [adjW], [adjH])

In [33]:
-- implement simple version of up-convolution
function upConvolution(net, d1, d2)
    net:add(nn.SpatialZeroPadding(0, 1, 0, 1))
    net:add(nn.SpatialFullConvolution(d1, d2, 5, 5, 2, 2, 2, 2))
    net:add(nn.SpatialZeroPadding(0, -1, 0, -1))
    
    net:add(nn.ReLU())
end

In [34]:
-- implement simple version of up-projection
function upProjection(net, d1, d2)
    local cat = nn.ConcatTable()
    
    local branch1 = nn.Sequential()
    branch1:add(nn.SpatialZeroPadding(0, 1, 0, 1))
    branch1:add(nn.SpatialFullConvolution(d1, d2, 5, 5, 2, 2, 2, 2))
    branch1:add(nn.SpatialZeroPadding(0, -1, 0, -1))
    branch1:add(nn.ReLU())
    branch1:add(nn.SpatialConvolution(d2, d2, 3, 3, 1, 1, 1, 1))
    
    local branch2 = nn.Sequential()
    branch2:add(nn.SpatialZeroPadding(0, 1, 0, 1))
    branch2:add(nn.SpatialFullConvolution(d1, d2, 5, 5, 2, 2, 2, 2))
    branch2:add(nn.SpatialZeroPadding(0, -1, 0, -1))
    
    cat:add(branch1)
    cat:add(branch2)
    net:add(cat)
    net:add(nn.CAddTable())
    
    net:add(nn.ReLU())
end

### The implementation of up_projection

In [35]:
-- input size 304x228x3
-- for the first part, we use pre-trained ResNet-50 built by facebook

-- Second step : different kinds of up-projection implementation

--build up projection blocks
up_projection = nn.Sequential()
upProjection(up_projection, 1024, 512)
upProjection(up_projection, 512, 256)
upProjection(up_projection, 256, 128)
upProjection(up_projection, 128, 64)

net = nn.Sequential()

net:add(up_projection)

d_final = 64
net:add(nn.SpatialConvolution(d_final, 1, 3, 3, 1, 1, 1, 1))
net:add(nn.ReLU())

print('CRN net\n' .. net:__tostring())

CRN net
nn.Sequential {
  [input -> (1) -> (2) -> (3) -> output]
  (1): nn.Sequential {
    [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> (9) -> (10) -> (11) -> (12) -> output]
    (1): nn.ConcatTable {
      input
        |`-> (1): nn.Sequential {
        |      [input -> (1) -> (2) -> (3) -> (4) -> (5) -> output]
        |      (1): nn.SpatialZeroPadding(l=0, r=1, t=0, b=1)
        |      (2): nn.SpatialFullConvolution(1024 -> 512, 5x5, 2,2, 2,2)
        |      (3): nn.SpatialZeroPadding(l=0, r=-1, t=0, b=-1)
        |      (4): nn.ReLU
        |      (5): nn.SpatialConvolution(512 -> 512, 3x3, 1,1, 1,1)
        |    }
         `-> (2): nn.Sequential {
               [input -> (1) -> (2) -> (3) -> output]
               (1): nn.SpatialZeroPadding(l=0, r=1, t=0, b=1)
               (2): nn.SpatialFullConvolution(1024 -> 512, 5x5, 2,2, 2,2)
               (3): nn.SpatialZeroPadding(l=0, r=-1, t=0, b=-1)
             }
         ... -> output
    }
    (2): nn.CAddTab

In [37]:
--test the network
test_tensor = torch.rand(1024, 10, 8)
print(#test_tensor)
test_output = net:forward(test_tensor)
print(#test_output)

 1024
   10
    8
[torch.LongStorage of size 3]



   1
 160
 128
[torch.LongStorage of size 3]

